1) Read in all files
2) See how many 0's, 1's
3) Analyze file (see the average length) 

In [10]:
import os

replay_path = "c:\\data\\label\\"
for dirName, subdirList, fileList in os.walk(replay_path):
    zeros = [f for f in fileList if f.split(".csv")[0].split("_",2)[2] == '0']
    ones = [f for f in fileList if f.split(".csv")[0].split("_",2)[2] == '1']
print(f"Zeros: {len(zeros)}\nOnes: {len(ones)}")
    

Zeros: 25
Ones: 25


In [22]:
import pandas as pd

all_files = zeros + ones
file_lengths = []
for f in all_files:
    df = pd.read_csv(os.path.join(replay_path,f))
    file_lengths.append(df.shape[0])
pd.Series(file_lengths).describe()


count     50.00000
mean     583.66000
std      125.65463
min      295.00000
25%      530.50000
50%      633.00000
75%      658.00000
max      776.00000
dtype: float64

4) For each file, take 5 seconds before and 1 second after the event of interest (sampling every 0.5 seconds)

In [58]:
from email import header


sample_interval = 0.5
seconds_before = 5
seconds_after = 1

all_seqs = []
for file in all_files:
    df = pd.read_csv(os.path.join(replay_path,file),index_col=0)
    min_dist = df['hausdorff_distance'].min()
    min_dist_idx = df['hausdorff_distance'].idxmin()
    start_idx = min_dist_idx - seconds_before * (2*sample_interval/0.05)
    stop_idx = min_dist_idx + seconds_after * (2*sample_interval/0.05)

    indices_to_sample = []
    for x in range(int(start_idx),int(stop_idx+1),int(sample_interval/0.05)): indices_to_sample.append(x)

    columns_of_interest = ['hausdorff_distance','angle',
        'ego_vel_x','ego_vel_y','ego_vel_z','ego_accel_x','ego_accel_y','ego_accel_z','ego_ang_vel_x','ego_ang_vel_y','ego_ang_vel_z',
        'adv_vel_x','adv_vel_y','adv_vel_z','adv_accel_x','adv_accel_y','adv_accel_z','adv_ang_vel_x','adv_ang_vel_y','adv_ang_vel_z']
    rows_of_interest = df.loc[indices_to_sample,columns_of_interest]
    one_seq = []
    for index, rows in rows_of_interest.iterrows():
        for elem in rows:
            one_seq.append(elem)
    one_seq.append(file.split(".csv")[0].split("_",2)[2])
    all_seqs.append(one_seq)

full_df = pd.DataFrame(all_seqs, columns=(columns_of_interest * 13 + ['class']))
full_df.to_csv("c:\\data\\labeled_data.csv",index=None)

5) Generate Train/Test Files

In [62]:
from sklearn.model_selection import train_test_split
x = full_df.iloc[:,:-1]
y = full_df.iloc[:,-1]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.33)
train = x_train.assign(Class=y_train)
test = x_test.assign(Class=y_test)
train.to_csv('c:\\data\\train_file.csv',index=None)
test.to_csv('c:\\data\\test_file.csv',index=None)